In [ ]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
import numpy as np

class SMME:
    def __init__(self, n_estimators=10):
        self.n_estimators = n_estimators
        self.estimators = [DecisionTreeClassifier(max_depth=1) for _ in range(n_estimators)] 

    def fit(self, x, y):
        n = x.shape[0]
        weights = np.array([1 / n] * n)
        indices = list(range(n))
        self.categories = len(np.unique(y))
        self.alphas = [0] * self.n_estimators
        def get_dataset_from(weights):
            new_indices = np.random.choice(indices, size=n, replace=True, p=weights)
            return x[new_indices], y[new_indices]
        x_train = x
        y_train = y
        for m in range(self.n_estimators):
            self.estimators[m].fit(x_train, y_train)
            predictions = self.estimators[m].predict(x)
            err = np.dot(weights, predictions != y) / np.sum(weights)
            alpha = np.log((1- err) / err) + np.log(self.categories - 1)
            weights = weights * np.exp(alpha * (predictions != y))
            weights /= np.sum(weights)
            x_train, y_train = get_dataset_from(weights)
            self.alphas[m] = alpha
    
    def predict(self, x):
        def weighetd_prediction(k):
            prediction = 0
            for m in range(self.n_estimators):
                p = self.estimators[m].predict([x])
                prediction += self.alphas[m] * (p[0] == k)
            return prediction
        predicted_class = None
        maximum = -np.inf
        for k in range(self.categories):
            wp = weighetd_prediction(k)
            if wp > maximum: 
                maximum = wp
                predicted_class = k
        return predicted_class



In [111]:
np.random.seed(0)
x, y = datasets.load_wine(return_X_y=True)
adaboost = SMME(10)
adaboost.fit(x, y)

In [112]:
ok = 0

for x_, y_ in zip(x, y):
    if adaboost.predict(x_) == y_:
        ok += 1

print(ok, y.shape[0])

176 178
